In [6]:
from sqlalchemy import create_engine
import pandas as pd


def connect_to_db():
    user = "root"
    password = ""   # sua senha, se tiver
    host = "localhost"
    port = "3306"
    database = "projetos"

    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"

    try:
        engine = create_engine(connection_string)
        print("✅ Conectado ao banco de dados!")
        return engine
    except Exception as e:
        print(f"❌ Erro na conexão: {e}")
        return None


In [7]:
QUERY_CARTEIRA = """
    SELECT
        DATA_BASE, UF, CODMUN, MUNICIPIO, 160_OPERACOES_DE_CREDITO, NOME_INSTITUICAO,
        AGEN_ESPERADAS, AGEN_PROCESSADAS, 161_EMPRES_E_TIT_DESCONTADOS,
        162_FINANCIAMENTOS, 163_FIN_RURAIS_AGRICUL_CUST_INVEST,
        167_FINANCIAMENTOS_AGROINDUSTRIAIS, 169_FINANCIAMENTOS_IMOBILIARIOS,
        171_OUTRAS_OPERACOES_DE_CREDITO, 172_OUTROS_CREDITOS,
        174_PROV_P_OPER_CREDITOS
    FROM projetos.dados_financeiros;
"""

In [8]:
def carregar_carteira():
    engine = connect_to_db()
    if engine is not None:
        df = pd.read_sql(QUERY_CARTEIRA, con=engine)
        print("✅ Dados carregados!")
        return df
    else:
        print("❌ Conexão falhou, dados não carregados.")
        return None


In [ ]:
df = carregar_carteira()
df['DATA_BASE'] = pd.to_datetime(df['DATA_BASE'], format='%Y%m')

df

✅ Conectado ao banco de dados!
✅ Dados carregados!


,DATA_BASE,UF,CODMUN,MUNICIPIO,160_OPERACOES_DE_CREDITO,NOME_INSTITUICAO,AGEN_ESPERADAS,AGEN_PROCESSADAS,161_EMPRES_E_TIT_DESCONTADOS,162_FINANCIAMENTOS,163_FIN_RURAIS_AGRICUL_CUST_INVEST,167_FINANCIAMENTOS_AGROINDUSTRIAIS,169_FINANCIAMENTOS_IMOBILIARIOS,171_OUTRAS_OPERACOES_DE_CREDITO,172_OUTROS_CREDITOS,174_PROV_P_OPER_CREDITOS
0,2023-12-01,MG,1690,CAMPESTRE,149353000.0,BCO DO BRASIL S.A.,1,1,16545600.0,381147.0,124435000.0,57275.0,4128690.0,41714.0,3764310.0,0.0
1,2023-12-01,MG,1690,CAMPESTRE,142214000.0,CAIXA ECONOMICA FEDERAL,1,1,15160600.0,136072.0,2540490.0,0.0,47370600.0,296874.0,78703500.0,-1993970.0
2,2023-12-01,MG,1690,CAMPESTRE,14015300.0,ITAÚ UNIBANCO S.A.,1,1,13884600.0,107719.0,0.0,0.0,0.0,0.0,76264.0,-53264.0
3,2023-12-01,MG,1690,CAMPESTRE,2875600.0,BCO BRADESCO S.A.,1,1,1175930.0,0.0,1481370.0,0.0,0.0,0.0,218299.0,0.0
4,2023-12-01,MG,3887,MONTE SANTO DE MINAS,169024000.0,BCO DO BRASIL S.A.,1,1,22777700.0,212542.0,91354300.0,2.0,2628040.0,88597.0,51963100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722,2024-12-01,SP,23771,SAO JOAO DA BOA VISTA,106068000.0,BCO SANTANDER (BRASIL) S.A.,2,2,83516900.0,11050200.0,24695100.0,0.0,0.0,0.0,587495.0,-13781900.0
2723,2024-12-01,SP,25470,DIVINOLANDIA,145952000.0,BCO DO BRASIL S.A.,1,1,18276700.0,90628.0,113097000.0,59.0,2042630.0,40078.0,12404900.0,0.0
2724,2024-12-01,SP,26943,ITOBI,4024300.0,BCO SANTANDER (BRASIL) S.A.,1,1,4447610.0,0.0,9495.0,0.0,0.0,0.0,46698.0,-479496.0
2725,2024-12-01,SP,37468,TAPIRATIBA,61303800.0,BCO DO BRASIL S.A.,1,1,25859300.0,536301.0,23680400.0,0.0,4794770.0,0.0,6433020.0,0.0


In [11]:
# Exemplo de um gráfico simples
import plotly.express as px

fig = px.line(df, x="DATA_BASE", y="162_FINANCIAMENTOS", title="Contratos por município")

In [12]:
df_agg = df.groupby(['DATA_BASE', 'MUNICIPIO'])['162_FINANCIAMENTOS'].sum().reset_index()


In [34]:
import plotly.graph_objects as go

# Agrupar dados
df_agg = df.groupby(['DATA_BASE', 'NOME_INSTITUICAO', 'MUNICIPIO'])['162_FINANCIAMENTOS'].sum().reset_index()

# Lista de municípios únicos
municipios = df_agg['MUNICIPIO'].unique()

# Definir cores
cores_bancos = {
    'BCO BRADESCO S.A.': '#CC092F',
    'BCO SANTANDER (BRASIL) S.A.': '#ED1C24',
    'ITAÚ UNIBANCO S.A.': '#FF6600',
    'CAIXA ECONOMICA FEDERAL': '#005CA9',
    'BCO DO BRASIL S.A.': '#FFCC00',
    'BCO SAFRA S.A.': '#002855',
    'BCO MERCANTIL DO BRASIL S.A.': '#0072C6',
    'BRB - BCO DE BRASILIA S.A.': '#003366',
    'BCO INTER': '#FF6A13',
    'BANCO GENIAL': '#6E2CA5',
    'BANCO BTG PACTUAL S.A.': '#002776',
    'BANCO MASTER': '#A71930',
    'BCO DAYCOVAL S.A': '#012D74',
    'BCO CEDULA S.A.': '#FF9900',
}

# Criar figura vazia
fig = go.Figure()

# Adicionar uma linha para cada instituição dentro de cada município
for municipio in municipios:
    df_municipio = df_agg[df_agg['MUNICIPIO'] == municipio]
    instituicoes = df_municipio['NOME_INSTITUICAO'].unique()

    for instituicao in instituicoes:
        df_inst = df_municipio[df_municipio['NOME_INSTITUICAO'] == instituicao]

        cor = cores_bancos.get(instituicao, None)  # Pega a cor definida ou deixa padrão

        fig.add_trace(go.Scatter(
            x=df_inst['DATA_BASE'],
            y=df_inst['162_FINANCIAMENTOS'],
            mode='lines+markers',
            name=f"{instituicao} ({municipio})",
            line=dict(color=cor),
            visible=True if municipio == municipios[0] else False
        ))

# Criar os botões do dropdown
buttons = []

for municipio in municipios:
    visible = []
    for m in municipios:
        df_m = df_agg[df_agg['MUNICIPIO'] == m]
        n_traces = df_m['NOME_INSTITUICAO'].nunique()
        if m == municipio:
            visible.extend([True] * n_traces)
        else:
            visible.extend([False] * n_traces)

    buttons.append(dict(
        label=municipio,
        method="update",
        args=[{"visible": visible},
              {"title": f"Evolução dos financiamentos - {municipio}"}]
    ))

# Botão para mostrar todos
buttons.append(dict(
    label="Mostrar Todos",
    method="update",
    args=[{"visible": [True] * len(fig.data)},
          {"title": "Evolução dos financiamentos por município"}]
))

# Layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0,
        y=1.15,
        xanchor='left',
        yanchor='top'
    )],
    xaxis_title="Data",
    yaxis_title="Valor (R$)",
    xaxis=dict(tickformat="%b/%Y"),
    template="plotly_white",
    title="Evolução dos financiamentos"
)

fig.show()


In [14]:
df_agg

,DATA_BASE,NOME_INSTITUICAO,MUNICIPIO,162_FINANCIAMENTOS
0,2023-12-01,BANCO ABN AMRO CLEARING S.A.,SAO PAULO,0.000000e+00
1,2023-12-01,BANCO BMG CONSIGNADO S.A.,SAO PAULO,0.000000e+00
2,2023-12-01,BANCO BTG PACTUAL S.A.,BELO HORIZONTE,0.000000e+00
3,2023-12-01,BANCO BTG PACTUAL S.A.,RIO DE JANEIRO,2.469170e+09
4,2023-12-01,BANCO BTG PACTUAL S.A.,SAO PAULO,1.235030e+09
...,...,...,...,...
2722,2024-12-01,OMNI BANCO S.A.,SAO PAULO,1.827740e+08
2723,2024-12-01,OURIBANK S.A.,SAO PAULO,6.835920e+07
2724,2024-12-01,PICPAY BANK - BANCO MÚLTIPLO S.A,SAO PAULO,0.000000e+00
2725,2024-12-01,SCOTIABANK BRASIL,SAO PAULO,1.253200e+09


In [31]:
# Limpar espaços
df_agg['MUNICIPIO'] = df_agg['MUNICIPIO'].str.strip().str.upper()

# Lista de municípios a excluir
excluir = ["SAO PAULO", "RIO DE JANEIRO", "BELO HORIZONTE"]

# Aplicar filtro
df_agg_tratado = df_agg[~df_agg['MUNICIPIO'].isin(excluir)]

df_agg_tratado

,DATA_BASE,NOME_INSTITUICAO,MUNICIPIO,162_FINANCIAMENTOS
42,2023-12-01,BCO BRADESCO S.A.,AGUAS DA PRATA,0.0
43,2023-12-01,BCO BRADESCO S.A.,ALFENAS,0.0
44,2023-12-01,BCO BRADESCO S.A.,ANDRADAS,0.0
46,2023-12-01,BCO BRADESCO S.A.,CACONDE,0.0
47,2023-12-01,BCO BRADESCO S.A.,CAMPESTRE,0.0
...,...,...,...,...
2714,2024-12-01,ITAÚ UNIBANCO S.A.,MONTE SANTO DE MINAS,16457.0
2715,2024-12-01,ITAÚ UNIBANCO S.A.,MUZAMBINHO,81223.0
2716,2024-12-01,ITAÚ UNIBANCO S.A.,POCOS DE CALDAS,2526980.0
2718,2024-12-01,ITAÚ UNIBANCO S.A.,SAO JOAO DA BOA VISTA,0.0


In [32]:
print (df_agg_tradado["NOME_INSTITUICAO"].unique())

['BANCO BTG PACTUAL S.A.' 'BANCO GENIAL' 'BANCO INTER' 'BANCO MASTER'
 'BANCO SEMEAR' 'BANCO SICOOB S.A.' 'BCO ABC BRASIL S.A.' 'BCO ALFA S.A.'
 'BCO ARBI S.A.' 'BCO BMG S.A.' 'BCO BOCOM BBM S.A.' 'BCO BRADESCO S.A.'
 'BCO BS2 S.A.' 'BCO CCB BRASIL S.A.' 'BCO CEDULA S.A.'
 'BCO CITIBANK S.A.' 'BCO CLASSICO S.A.' 'BCO DAYCOVAL S.A'
 'BCO DO BRASIL S.A.' 'BCO DO ESTADO DO RS S.A.'
 'BCO DO NORDESTE DO BRASIL S.A.' 'BCO GUANABARA S.A.'
 'BCO INDUSTRIAL DO BRASIL S.A.' 'BCO ITAÚ BBA S.A.'
 'BCO J.P. MORGAN S.A.' 'BCO MERCANTIL DO BRASIL S.A.' 'BCO MODAL S.A.'
 'BCO RENDIMENTO S.A.' 'BCO SAFRA S.A.' 'BCO SANTANDER (BRASIL) S.A.'
 'BCO SOFISA S.A.' 'BCO XP S.A.' 'BNY MELLON BCO S.A.'
 'BRB - BCO DE BRASILIA S.A.' 'CAIXA ECONOMICA FEDERAL'
 'ITAÚ UNIBANCO S.A.' 'BOC BRASIL' 'BCO FATOR S.A.']


In [9]:
from pathlib import Path
import requests
import sidrapy

def get_periodos(agregado: str):
    url = f"https://servicodados.ibge.gov.br/api/v3/agregados/{agregado}/periodos"
    response = requests.get(url)
    return response.json()

def download_table(
    sidra_tabela: str,
    territorial_level: str,
    ibge_territorial_code: str,
    variable: str = "allxp",
    classifications: dict = None,
    data_dir: Path = Path("data"),
) -> list[Path]:
    """Download a SIDRA table in CSV format on temp_dir()

    Args:
        sidra_tabela (str): SIDRA table code
        territorial_level (str): territorial level code
        ibge_territorial_code (str): IBGE territorial code
        variable (str, optional): variable code. Defaults to None.
        classifications (dict, optional): classifications and categories codes.
            Defaults to None.

    Returns:
        list[Path]: list of downloaded files
    """
    filepaths = []
    periodos = get_periodos(sidra_tabela)
    for periodo in periodos:
        filename = f"{periodo['id']}.csv"
        dest_filepath = data_dir / filename
        dest_filepath.parent.mkdir(exist_ok=True, parents=True)
        if dest_filepath.exists():
            print("File already exists:", dest_filepath)
            continue
        print("Downloading", filename)
        df = sidrapy.get_table(
            table_code=sidra_tabela,  # Tabela SIDRA
            territorial_level=territorial_level,  # Nível de Municípios
            ibge_territorial_code=ibge_territorial_code,  # Territórios
            period=periodo["id"],  # Período
            variable=variable,  # Variáveis
            classifications=classifications,
        )
        df.to_csv(dest_filepath, index=False, encoding="utf-8")
        filepaths.append(dest_filepath)
    return filepaths


In [10]:
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

files = []

# Populacao Censos
sidra_tabela = "200"
territorial_level = "6"
ibge_territorial_code = "all"

files_census = download_table(
    sidra_tabela=sidra_tabela,
    territorial_level=territorial_level,
    ibge_territorial_code=ibge_territorial_code,
    variable="allxp",
    classifications={"2": "0", "1": "0", "58": "0"},
    data_dir=data_dir,
)
files.extend(files_census)

# Populacao Censo 2022
sidra_tabela = "9514"
territorial_level = "6"
ibge_territorial_code = "all"

files_census_2022 = download_table(
    sidra_tabela=sidra_tabela,
    territorial_level=territorial_level,
    ibge_territorial_code=ibge_territorial_code,
    variable="allxp",
    classifications={"2": "6794", "287": "100362", "286": "113635"},
    data_dir=data_dir,
)
files.extend(files_census_2022)

# Populacao Contagens
sidra_tabelas = (
    "305",
    "793",
)

for sidra_tabela in sidra_tabelas:
    files_counts = download_table(
        sidra_tabela=sidra_tabela,
        territorial_level=territorial_level,
        ibge_territorial_code=ibge_territorial_code,
        data_dir=data_dir,
    )
    files.extend(files_counts)

# Populacao Estimativas
sidra_tabela = "6579"

files_estimates = download_table(
    sidra_tabela=sidra_tabela,
    territorial_level=territorial_level,
    ibge_territorial_code=ibge_territorial_code,
    data_dir=data_dir,
)
files.extend(files_estimates)


In [11]:
import pandas as pd

def read_file(filepath: Path, **read_csv_args) -> pd.DataFrame:
    print("Reading file", filepath)
    data = pd.read_csv(filepath, skiprows=1, na_values=["...", "-"], **read_csv_args)
    data = data.dropna(subset="Valor")
    return data

def refine(df: pd.DataFrame) -> pd.DataFrame:
    df = (
        df.dropna(subset="Valor")
        .rename(
            columns={
                "Ano": "ano",
                "Município (Código)": "id_municipio",
                "Valor": "pessoas",
            }
        )
        .assign(pessoas=lambda x: x["pessoas"].astype(int))
    )
    df[["nome_municipio", "sigla_uf"]] = df["Município"].str.split(" - ", expand=True)
    df = df.drop(columns="Município")
    df = df[["ano", "id_municipio", "nome_municipio", "sigla_uf", "pessoas"]]
    return df


df = refine(
    pd.concat(
        (
            read_file(file, usecols=("Ano", "Município (Código)", "Município", "Valor"))
            for file in files
        ),
        ignore_index=True,
    )
)


Reading file data\1970.csv
Reading file data\1980.csv
Reading file data\1991.csv
Reading file data\2000.csv
Reading file data\2010.csv
Reading file data\2022.csv
Reading file data\1996.csv
Reading file data\2007.csv
Reading file data\2001.csv
Reading file data\2002.csv
Reading file data\2003.csv
Reading file data\2004.csv
Reading file data\2005.csv
Reading file data\2006.csv
Reading file data\2008.csv
Reading file data\2009.csv
Reading file data\2011.csv
Reading file data\2012.csv
Reading file data\2013.csv
Reading file data\2014.csv
Reading file data\2015.csv
Reading file data\2016.csv
Reading file data\2017.csv
Reading file data\2018.csv
Reading file data\2019.csv
Reading file data\2020.csv
Reading file data\2021.csv
Reading file data\2024.csv


In [1]:
import pandas as pd
import random

# Gerar 12 números aleatórios entre 10.000.000 e 15.000.000
numeros_aleatorios = [random.randint(10_000_000, 15_000_000) for _ in range(12)]

# Criar o DataFrame
df = pd.DataFrame(numeros_aleatorios, columns=['Número'])

# Exibir o DataFrame
print(df)


      Número
0   11762033
1   14682440
2   14245004
3   10647882
4   11597292
5   14113382
6   10744094
7   13776896
8   14814956
9   11232684
10  10281816
11  12874164
